<a href="https://colab.research.google.com/github/wentaosu915/atkinson/blob/main/Atkinson_index_in_A_B_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import norm

def atkinson_index(values, epsilon=0.5):
    """Compute the Atkinson Index based on section 2.2 of the paper."""
    values = np.array(values)
    mean_value = np.mean(values)

    if epsilon == 1:
        return 1 - np.exp(np.mean(np.log(values / mean_value)))
    else:
        return 1 - (np.mean((values / mean_value) ** (1 - epsilon))) ** (1 / (1 - epsilon))

def atkinson_variance(values, epsilon=0.5):
    """Compute the variance of the Atkinson Index using formula (5) from the paper."""
    values = np.array(values)
    n = len(values)

    mean_value = np.mean(values)
    x_power_1_minus_e = np.mean(values ** (1 - epsilon))
    x_power_2_minus_2e = np.mean(values ** (2 - 2 * epsilon))
    x_power_2 = np.mean(values ** 2)
    x_power_2_minus_e = np.mean(values ** (2 - epsilon))

    sigma_11 = x_power_2_minus_2e - x_power_1_minus_e ** 2
    sigma_12 = x_power_2_minus_e - mean_value * x_power_1_minus_e
    sigma_22 = x_power_2 - mean_value ** 2

    term1 = sigma_11 * (x_power_1_minus_e ** ((2 * epsilon) / (1 - epsilon))) / ((1 - epsilon) ** 2 * mean_value ** 2)
    term2 = -2 * sigma_12 * (x_power_1_minus_e ** ((1 + epsilon) / (1 - epsilon))) / ((1 - epsilon) * mean_value ** 3)
    term3 = sigma_22 * (x_power_1_minus_e ** (2 / (1 - epsilon))) / (mean_value ** 4)

    variance = term1 + term2 + term3
    return variance

def compute_atkinson_lift_pvalue(df, epsilon=0.2):
    """
    Compute the Atkinson Index lift and p-value from an A/B test dataframe.

    Args:
    - df (pd.DataFrame): DataFrame containing 'treatment' (binary 0/1), 'metric_value', and 'user_id'.
    - epsilon (float): Inequality aversion parameter.

    Returns:
    - atkinson_control (float): Atkinson Index for the control group.
    - atkinson_treatment (float): Atkinson Index for the treatment group.
    - observed_lift (float): Lift in Atkinson Index.
    - p_value (float): p-value for statistical significance.
    """
    # Split the data into control and treatment groups
    control_values = df[df['treatment'] == 0]['metric_value'].values
    treatment_values = df[df['treatment'] == 1]['metric_value'].values

    # Compute Atkinson Index for each group
    atkinson_control = atkinson_index(control_values, epsilon)
    atkinson_treatment = atkinson_index(treatment_values, epsilon)

    # Compute the observed lift
    observed_lift = atkinson_treatment - atkinson_control

    # Compute the variances using the corrected delta method formula
    var_control = atkinson_variance(control_values, epsilon)
    var_treatment = atkinson_variance(treatment_values, epsilon)

    # Compute standard error of the lift
    std_error_lift = np.sqrt(var_control / len(control_values) + var_treatment / len(treatment_values))

    # Compute p-value using the normal approximation
    z_score = observed_lift / std_error_lift
    p_value = 2 * (1 - norm.cdf(abs(z_score)))

    return atkinson_control, atkinson_treatment, observed_lift, p_value


In [3]:
# Example Usage
# np.random.seed(42)

# Generate synthetic A/B test data
num_users = 100000
df = pd.DataFrame({
    'user_id': np.arange(num_users),
    'treatment': np.random.choice([0, 1], size=num_users),  # Random assignment
    'metric_value': np.random.exponential(scale=50, size=num_users)  # Example skewed distribution
})

# Compute Atkinson Index lift and p-value
atkinson_control, atkinson_treatment, observed_lift, p_value = compute_atkinson_lift_pvalue(df, epsilon=0.2)

print(f"Atkinson Index (Control): {atkinson_control:.4f}")
print(f"Atkinson Index (Treatment): {atkinson_treatment:.4f}")
print(f"Observed Lift in Atkinson Index: {observed_lift:.4f}")
print(f"p-value: {p_value:.4f}")


Atkinson Index (Control): 0.0854
Atkinson Index (Treatment): 0.0853
Observed Lift in Atkinson Index: -0.0001
p-value: 0.8303


**AA validation based on randomly assigned users**

In [4]:
# Example Usage
# np.random.seed(42)
p_values = []
for i in range(1000):
  # Generate synthetic A/B test data
  num_users = 1000000
  df = pd.DataFrame({
      'user_id': np.arange(num_users),
      'treatment': np.random.choice([0, 1], size=num_users),  # Random assignment
      'metric_value': np.random.exponential(scale=50, size=num_users)  # Example skewed distribution
  })

  # Compute Atkinson Index lift and p-value
  atkinson_control, atkinson_treatment, observed_lift, p_value = compute_atkinson_lift_pvalue(df, epsilon=0.2)
  p_values.append(p_value)

df_p_values = pd.DataFrame({'p_value': p_values})

# Calculate the proportion of p-values less than 0.05
proportion = (df_p_values['p_value'] < 0.05).mean()

# Print the proportion
proportion

np.float64(0.039)

**Test the power of Atkinson index statistical inference based on variance from delta method**

In [ ]:
# prompt: can we make the treatment users metric_value distributed more uneven than control users?


import numpy as np
import pandas as pd
from scipy.stats import norm
num_users = 10000000

p_values = []
for i in range(100):
# Generate synthetic A/B test data with uneven distribution for treatment
  df = pd.DataFrame({
      'user_id': np.arange(num_users),
      'treatment': np.random.choice([0, 1], size=num_users),  # Random assignment
      'metric_value': np.random.exponential(scale=50, size=num_users)  # Example skewed distribution
  })

  # Modify the metric_value for treatment users to be more uneven
  treatment_users = df[df['treatment'] == 1].index
  num_treatment_users = len(treatment_users)
  # Create a more uneven distribution, e.g., by adding a large value to a small portion of users
  df.loc[treatment_users[:int(0.01 * num_treatment_users)], 'metric_value'] += 100

  # Compute Atkinson Index lift and p-value
  atkinson_control, atkinson_treatment, observed_lift, p_value = compute_atkinson_lift_pvalue(df, epsilon=0.2)
  p_values.append(p_value)

df_p_values = pd.DataFrame({'p_value': p_values})

# Calculate the proportion of p-values less than 0.05
proportion = (df_p_values['p_value'] < 0.05).mean()

# Print the proportion
proportion



In [6]:
 df = pd.DataFrame({
      'user_id': np.arange(num_users),
      'treatment': np.random.choice([0, 1], size=num_users),  # Random assignment
      'metric_value': np.random.exponential(scale=50, size=num_users)  # Example skewed distribution
  })
df.describe()

,user_id,treatment,metric_value
count,1000000.000000,1000000.000000,1000000.000000
mean,499999.500000,0.501135,49.901778
std,288675.278933,0.499999,49.981712
min,0.000000,0.000000,0.000013
25%,249999.750000,0.000000,14.346211
50%,499999.500000,1.000000,34.563302
75%,749999.250000,1.000000,69.149613
max,999999.000000,1.000000,744.855368
